In [1]:
!pip3 install langchain-gigachat langchain-chroma langchain-core pandas numpy

In [2]:
!pip3 install openai

In [3]:
import pandas as pd
import numpy as np
from openai import OpenAI
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings
from langchain_gigachat.chat_models import GigaChat
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import ast, os


In [4]:
def load_embeddings_to_dataframe(filepath):
    """
    Загружает данные эмбеддингов из CSV файла в pandas DataFrame
    
    Args:
        filepath (str): Путь к файлу с данными
        
    Returns:
        pd.DataFrame: DataFrame с загруженными данными
    """
    try:
        df = pd.read_csv(filepath, on_bad_lines='skip')
        print(f"Загружено {len(df)} записей из файла {filepath}")
        print(f"Колонки: {df.columns.tolist()}")
        print(f"\nПервые 3 записи:")
        print(df.head(3))
        return df
    except Exception as e:
        print(f"Произошла ошибка при загрузке данных: {e}")
        return None

# Загрузка данных
df = load_embeddings_to_dataframe("arxiv_embeddings202505211515.csv")


Загружено 50000 записей из файла arxiv_embeddings202505211515.csv
Колонки: ['article_id', 'submitter', 'authors', 'title', 'comments', 'journal_ref', 'doi', 'report_no', 'categories', 'main_category', 'other_categories', 'license', 'abstract', 'versions', 'authors_parsed', 'year', 'embedding_bge_m3']

Первые 3 записи:
     article_id submitter                                            authors  \
0  2410.08001v2       NaN  {"Qingwen Bu","Hongyang Li","Li Chen","Jisong ...   
1  2410.11878v1       NaN                     {"Xingyi Yang","Xinchao Wang"}   
2  2410.07991v3       NaN  {"Tommaso Giorgi","Lorenzo Cima","Tiziano Fagn...   

                                               title comments journal_ref  \
0  Towards Synergistic, Generalized, and Efficien...      NaN         NaN   
1                               Neural Metamorphosis      NaN         NaN   
2  Human and LLM Biases in Hate Speech Annotation...      NaN         NaN   

   doi report_no           categories main_categor

In [5]:
print(f"Размер датасета: {df.shape}")
print(f"\nТипы данных:\n{df.dtypes}")
print(f"\nПроверка пропущенных значений:\n{df.isnull().sum()}")


Размер датасета: (50000, 17)

Типы данных:
article_id          object
submitter           object
authors             object
title               object
comments            object
journal_ref         object
doi                 object
report_no           object
categories          object
main_category       object
other_categories    object
license             object
abstract            object
versions            object
authors_parsed      object
year                 int64
embedding_bge_m3    object
dtype: object

Проверка пропущенных значений:
article_id              0
submitter            9197
authors                 0
title                   0
comments            23907
journal_ref         43700
doi                 43125
report_no           47991
categories              0
main_category           0
other_categories     6907
license              9363
abstract                0
versions             9197
authors_parsed       9197
year                    0
embedding_bge_m3        0
dtype: int

In [6]:
def prepare_documents(df, limit=1000):
    """
    Преобразует DataFrame в список документов LangChain
    
    Args:
        df: DataFrame с данными статей
        limit: Количество документов для обработки
        
    Returns:
        List[Document]: Список документов
    """
    documents = []
    
    # Возьмите первые limit записей
    df_subset = df.head(limit)
    
    for idx, row in df_subset.iterrows():
        # Формируйте текст документа из доступных полей
        page_content = ""
        
        # Адаптируйте под структуру вашего датасета
        if 'title' in row and pd.notna(row['title']):
            page_content += f"Название: {row['title']}\n"
        if 'abstract' in row and pd.notna(row['abstract']):
            page_content += f"Аннотация: {row['abstract']}\n"
        if 'authors' in row and pd.notna(row['authors']):
            page_content += f"Авторы: {row['authors']}\n"
        
        # Метаданные
        metadata = {}
        if 'category' in row and pd.notna(row['category']):
            metadata['category'] = row['category']
        if 'date' in row and pd.notna(row['date']):
            metadata['date'] = row['date']
        if 'arxiv_id' in row and pd.notna(row['arxiv_id']):
            metadata['arxiv_id'] = row['arxiv_id']
        
        if page_content.strip():  # Добавляем только непустые документы
            documents.append(Document(
                page_content=page_content,
                metadata=metadata
            ))
    
    print(f"Подготовлено {len(documents)} документов")
    return documents

# Подготовка документов
documents = prepare_documents(df, limit=1000)

# Просмотр примера документа
if documents:
    print("Пример документа:")
    print(f"Содержимое: {documents[0].page_content[:200]}...")
    print(f"Метаданные: {documents[0].metadata}")
else:
    print("Документы не загружены")

Подготовлено 1000 документов
Пример документа:
Содержимое: Название: Towards Synergistic, Generalized, and Efficient Dual-System for Robotic Manipulation
Аннотация: The increasing demand for versatile robotic systems to operate in diverse and
dynamic environm...
Метаданные: {}


In [7]:
client = OpenAI(     
api_key="ZmJhMjUwZTItMDg0ZC00N2E3LWIyNDktYjA4MTQyZGFmMGE4.97f6d089a16317c3aa93b365eda739a8",
    base_url="https://foundation-models.api.cloud.ru/v1"
)


def get_embedding(text: str, model="BAAI/bge-m3") -> list:
    """Получает эмбеддинг текста"""
    response = client.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

# Тестирование функции эмбеддингов
test_embedding = get_embedding("Тестовый запрос")
print(f"Размерность эмбеддинга: {len(test_embedding)}")


Размерность эмбеддинга: 1024


In [ ]:
from langchain_core.embeddings import Embeddings
from typing import List

class CustomEmbeddings(Embeddings):
    """Кастомный класс эмбеддингов для работы с API"""
    
    def __init__(self, client, model="BAAI/bge-m3"):
        self.client = client
        self.model = model
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Получение эмбеддингов для списка документов"""
        embeddings = []
        for text in texts:
            response = self.client.embeddings.create(
                input=[text],
                model=self.model
            )
            embeddings.append(response.data[0].embedding)
        return embeddings
    
    def embed_query(self, text: str) -> List[float]:
        """Получение эмбеддинга для запроса"""
        response = self.client.embeddings.create(
            input=[text],
            model=self.model
        )
        return response.data[0].embedding

# Создание экземпляра эмбеддингов
embeddings = CustomEmbeddings(client)



: 

In [ ]:
print("Создание векторного хранилища...")
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="arxiv_papers",
    persist_directory="./chroma_db"  # Директория для сохранения
)
print("Векторное хранилище успешно создано!")

# Проверка работы хранилища
test_query = "машинное обучение и нейронные сети"
results = vectorstore.similarity_search(test_query, k=3)
print(f"\nРезультаты поиска по запросу '{test_query}':")
for i, doc in enumerate(results, 1):
    print(f"\n{i}. {doc.page_content[:200]}...")
    print(f"   Метаданные: {doc.metadata}")

Создание векторного хранилища...
